<a href="https://colab.research.google.com/github/katiatel/ml/blob/main/all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import DistanceMetric

In [ ]:
DATA = pd.read_csv('pulsar_stars_new.csv', decimal='.', delimiter=",")

pulsar_stars = DATA[((DATA.TARGET == 0) & (DATA.MIP >= 83) & (DATA.MIP <= 84)) |
                    ((DATA.TARGET == 1) & (DATA.MIP >= 83) & (DATA.MIP <= 89))]

print("Кол-во строк:", len(pulsar_stars))
print("До нормировки:", pulsar_stars.MIP.mean())

Кол-во строк: 136
До нормировки: 84.54279641544117


In [ ]:
scaler = MinMaxScaler()
scaler.fit(pulsar_stars)
pulsar_stars = pd.DataFrame(scaler.transform(pulsar_stars), columns=pulsar_stars.columns)

print("После нормировки:", pulsar_stars.MIP.mean())

После нормировки: 0.26595638255302173


In [ ]:
STAR = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] #меняем числа


In [ ]:
X = pd.DataFrame(pulsar_stars.drop(["TARGET"], axis=1))
y = pd.DataFrame(pulsar_stars["TARGET"])

In [ ]:
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel()) #параметры вписать
for i, p in enumerate(reg.predict_proba([STAR])[0]):
    print("Вероятность отнесения к классу \"" + ("не пульсар" if i == 0 else "пульсар") + "\": ", p) #внимание пульсар или непульсар

Вероятность отнесения к классу "не пульсар":  0.7264450276675015
Вероятность отнесения к классу "пульсар":  0.2735549723324985


In [ ]:
for m in ["euclidean", "manhattan"]: #выбираем нужную метрику если не указано ничего не меняем
    dist = DistanceMetric.get_metric(m)
    distances = [dist.pairwise(np.concatenate(([i], [STAR])))[0][1] for i in list(X.values)]

In [ ]:
    print("Расстояние до ближайшего по метрике \"" + m + "\":", min(distances))

Расстояние до ближайшего по метрике "manhattan": 0.5478999978926339


задание 2

In [ ]:
!pip install mnist
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

In [ ]:
import pandas as pd
import numpy as np
import mnist
import warnings

In [ ]:
if __name__ == '__main__':
    exp_disp = 0.77 #данные для первого пункта
    test_ratio = 0.3 #значения из 3 пункта
    random_state = 48 #числа из 4п
    min_samples_leaf = 10 # числа из 4п
    max_depth = 20 # числа из 4п
    n_estimators = 10 # числа из 4п
    clazz_randomforest = 5 #4п объектов класса 5
    clazz_log_reg = 8
    clazz_tree = 6
    #image_file_randomforest = 5
    #image_file_log_reg = 5
    #image_file_tree = 5

In [ ]:
    from keras.datasets import mnist

    (X_train, y_train), (X_pred, y_pred) = mnist.load_data()
    dim = 784  # 28*28
    X_train_ = X_train.reshape(len(X_train), dim)

    pca = PCA(svd_solver='full')
    pca = pca.fit(X_train_)

    M = 0
    for arg, val in enumerate(np.cumsum(pca.explained_variance_ratio_)):
        if val > exp_disp:
            M = arg + 1
            break
    
    print("Количество главных компонент, чтобы доля объяснённой дисперсии превышала " + str(exp_disp) + ": " + str(M))

Количество главных компонент, чтобы доля объяснённой дисперсии превышала 0.77: 37


In [ ]:
    X_train = X_train.reshape(len(X_train), dim)

    # Поиск счёт
    pca = PCA(n_components=M, svd_solver='full')
    pca = pca.fit(X_train)

    # Разделение выборки
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_ratio, random_state=random_state)
    
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)

    print("Выборочное среднее нулевой колонки тренировочного набора: " + str(X_train.transpose()[0].mean()))

Выборочное среднее нулевой колонки тренировочного набора: 2.032837033185744


In [ ]:
    random_forest = RandomForestClassifier(criterion='gini', min_samples_leaf=min_samples_leaf, max_depth=max_depth, n_estimators=n_estimators, random_state=random_state)
    clf_random_forest = OneVsRestClassifier(random_forest).fit(X_train, y_train)
    y_pred = clf_random_forest.predict(X_test)

    CM = confusion_matrix(y_test, y_pred)
    print("Количество верно классифицированных объектов класса " + str(clazz_randomforest) + ": " + str(CM[clazz_randomforest][clazz_randomforest]))

Количество верно классифицированных объектов класса 5: 1441


In [ ]:
lr = LogisticRegression(random_state=48, solver='lbfgs')
clf = OneVsRestClassifier(lr).fit(X_train, y_train)
y_pred = clf.predict(X_test)

CM = confusion_matrix(y_test, y_pred)
print("Количество верно классифицированных объектов класса " + str(clazz_log_reg) + ": " + str(CM[clazz_log_reg][clazz_log_reg]))

Количество верно классифицированных объектов класса 8: 1481


In [ ]:
dtc = DecisionTreeClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, random_state=48)
clf = OneVsRestClassifier(dtc).fit(X_train, y_train)
y_pred = clf.predict(X_test)

CM = confusion_matrix(y_test, y_pred)
print("Количество верно классифицированных объектов класса " + str(clazz_tree) + ": " + str(CM[clazz_tree][clazz_tree]))

Количество верно классифицированных объектов класса 6: 1643
